In [ ]:
%pip install mediapipe

In [ ]:
%pip install pandas

In [ ]:
import cv2
import numpy as np
import pandas as pd
import mediapipe as mp
from PIL import Image
import os

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands

def process_video(input_path, output_path):
    """
    Process a video with Mediapipe Pose + Hands.
    Draws skeleton and hand landmarks, saves to output_path.
    """
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"❌ Could not open {input_path}")
        return

    # Setup output video writer
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS)
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    pose = mp_pose.Pose(static_image_mode=False, model_complexity=2)
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pose_results = pose.process(image_rgb)
        hand_results = hands.process(image_rgb)
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        # Draw pose
        if pose_results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image_bgr,
                pose_results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
            )

        # Draw hands
        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image_bgr,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)
                )

        out.write(image_bgr)

    pose.close()
    hands.close()
    cap.release()
    out.release()
    print(f"✅ Processed and saved: {output_path}")

def process_multiple_videos(input_folder, output_folder):
    """
    Runs Mediapipe processing on all videos in a folder.
    Saves outputs to output_folder with same filenames.
    """
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.mp4', '.mov', '.avi', '.mkv')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, f"processed_{filename}")
            process_video(input_path, output_path)

# Example usage:
if __name__ == "__main__":
    input_folder = "vids"           # your folder containing raw videos
    output_folder = "landmarked"  # folder for processed outputs
    process_multiple_videos(input_folder, output_folder)


In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands

# landmark indices from mediapipe pose model
FACE_POINTS = {
    "nose": 0,
    "left_eye": 2,
    "right_eye": 5,
    "mouth_left": 9,
    "mouth_right": 10
}

def process_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"❌ Could not open {input_path}")
        return

    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS)
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    pose = mp_pose.Pose(static_image_mode=False, 
                        model_complexity=2,
                        min_detection_confidence=0.2,   # lower threshold so it doesn’t skip blurry frames
                        min_tracking_confidence=0.2
                        )
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pose_results = pose.process(image_rgb)
        hand_results = hands.process(image_rgb)
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        # Draw pose landmarks (body)
        if pose_results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image_bgr,
                pose_results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
            )

            # Draw minimal face points (eyes, nose, mouth)
            h, w, _ = image_bgr.shape
            for name, idx in FACE_POINTS.items():
                lm = pose_results.pose_landmarks.landmark[idx]
                x, y = int(lm.x * w), int(lm.y * h)
                cv2.circle(image_bgr, (x, y), 5, (0, 255, 255), -1)
                # cv2.putText(image_bgr, name, (x + 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

        # Draw hands
        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    image_bgr,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)
                )

        out.write(image_bgr)

    pose.close()
    hands.close()
    cap.release()
    out.release()
    print(f"✅ Processed and saved: {output_path}")

def process_multiple_videos(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    # ip = ["00335.mp4","00"]
    
    
    # return
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.mp4', '.mov', '.avi', '.mkv')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, f"processed_{filename}")
            process_video(input_path, output_path)

# Example usage
if __name__ == "__main__":
    input_folder = "vids"           # your folder containing raw videos
    output_folder = "landmarked"  # folder for processed outputs
    process_multiple_videos(input_folder, output_folder)


✅ Processed and saved: landmarked\processed_00335.mp4
✅ Processed and saved: landmarked\processed_00336.mp4
✅ Processed and saved: landmarked\processed_00338.mp4
✅ Processed and saved: landmarked\processed_00339.mp4
